In [1]:
from glob import glob
import re
import json
def loadFile(file_name) :
    with open(file_name, "r") as f :
        return "".join(f.readlines())
    return ""

def saveFile(file_name, contents) :
    with open(file_name, "w") as f :
        f.write(contents)

def loadJson(json_file_path) :
    with open(json_file_path, encoding='utf-8') as json_file :
        return json.load(json_file)

simpleApi = loadJson("../ngx3js-module/docs_api.json")["children"]


In [2]:
def getClassInfo(className : str) :
    for info in simpleApi :
        if (info["name"] == className) :
            return info
    return None

def getSourceFile(info) :
    if "sources" in info :
        fileName : str = info["sources"][0]["fileName"]
        if fileName.startswith("src/app/three/") :
            return fileName[14:-3]
    return "error"

def getSameSourceFile(sorceUrl) :
    list = []
    for info in simpleApi :
        infoSource = getSourceFile(info)
        kindString = info["kindString"]
        if sorceUrl == infoSource and kindString != 'Class':
           list.append(info) 
    return list

def isDecorators(info, decoType) :
    isInput = False
    if 'decorators' in info :
        decorators = info['decorators']
        for deco in decorators :
            if deco['name'] == decoType :
                isInput = True
    return isInput

def getExtends(info) :
    extendeds = []
    if "extendedTypes" in info : 
        extendedTypeName = info["extendedTypes"][0]["name"]
        childInfo = getClassInfo(extendedTypeName)
        if childInfo is not None :
            parentExtends = getExtends(childInfo)
            if len(parentExtends) > 0 :
                for name in parentExtends :
                    extendeds.append(name)
        extendeds.append(extendedTypeName)
    return extendeds

def getComment(info) :
    commentHtml = ''
    comment = None
    commentTxt = []
    if "comment" in info :
        comment = info['comment']
    if comment is not None :
        if 'shortText' in comment and 'text' in comment :
            commentTxt.append(getCommentCode(comment['shortText'].strip()))
            commentTxt.append("")
            commentTxt.append(getCommentCode(comment['text'].strip()))
        elif 'shortText' in comment :
            commentTxt.append(getCommentCode(comment['shortText'].strip()))
        if 'tags' in comment :
            tagHtml = []
            for tags in comment['tags'] :
                if (tags['tag'] == 'see'):
                    tagText = tags['text'].strip()
                    if tagText.endswith(",") : 
                        tagText = tagText[0:-1]
                    threeSelector = re.compile("THREE\.([a-zA-Z0-9]+)")
                    matches = threeSelector.findall(tagText)
                    if matches :
                        tagHtml.append("[page:{0} {0}] - {1}".format(
                            matches[0],
                            tagText
                        ))
                    else :
                        tagHtml.append(tagText) 
            if len(tagHtml) > 0 :
                commentTxt.append("")
                commentTxt.append("\n".join(tagHtml).strip())
    if 'type' in info :
        childType = info['type']['type']
        if childType == 'union' :
            typeNames = []
            for typeInfo in info['type']['types'] :
                if 'name' in typeInfo :
                    typeNames.append('[page:{0} {0}]'.format(typeInfo['name']))
                elif typeInfo['type'] == 'array' and 'elementType' in typeInfo and 'name' in typeInfo['elementType']:
                    typeNames.append('[page:{0} {0}]'.format(typeInfo['elementType']['name']))
            if len(typeNames) > 0 :
                commentTxt.append("")
                commentTxt.append(" | ".join(typeNames).strip())
        elif childType == 'reflection' and 'declaration' in info['type']:
            declaration = info['type']['declaration']
            if 'indexSignature' in declaration :
                parameters = declaration['indexSignature']['parameters']
                key = parameters[0]['name']
                keyType = parameters[0]['type']['name']
                value = 'any'
                if 'name' in declaration['indexSignature']['type'] :
                    value = declaration['indexSignature']['type']['name']
                commentTxt.append("")
                commentTxt.append(" {{ [ {0} : [page:{1} {1}] ] : [page:{2} {2}] }} ".format(key, keyType, value ))
    if comment is not None :
        if 'returns' in comment :
            commentTxt.append("")
            commentTxt.append("Returns : {0}".format(comment['returns'].strip()))
    if len(commentTxt) > 0 :
        commentHtml = "\n".join(commentTxt)
        commentHtml = commentHtml.replace("<br>","").replace("<br/>","").replace("<br />","")
        commentHtml = commentHtml.replace("\n\n\n","\n\n")
        commentHtml = commentHtml.replace("\n","<br>\n\t\t\t").strip()
        
    return commentHtml

def getParamTypes(child) :
    typeName = 'any'
    if 'type' in child :
        childType = child['type']['type']
        if childType == 'union' :
            typeNames = []
            for typeInfo in child['type']['types'] :
                if 'name' in typeInfo :
                    typeNames.append(typeInfo['name'])
                elif typeInfo['type'] == 'array' and 'elementType' in typeInfo and 'name' in typeInfo['elementType']:
                    typeNames.append(typeInfo['elementType']['name'])
            typeName = '|'.join(typeNames)
            typeName = 'mixed'
        elif childType == 'array' : 
            if 'elementType' in child['type'] and 'name' in child['type']['elementType'] :
                typeName = child['type']['elementType']['name'] + "[]"
            #else  :
                #print("Error")
                #print(child['type'])
        elif 'name' in child['type']:
            typeName = child['type']['name']
    return typeName

def getGenApiDocs(className : str) :
    info = getClassInfo(className)
    if info is None :
        return False
    kindString = info["kindString"]
    html = []
    html.append('<!DOCTYPE html>')
    html.append('<html lang="en">')
    html.append('	<head>')
    html.append('		<meta charset="utf-8" />')
    html.append('		<base href="../../" />')
    html.append('		<script src="page.js"></script>')
    html.append('		<link type="text/css" rel="stylesheet" href="page.css" />')
    html.append('	</head>')
    html.append('	<body>')
    extends = getExtends(info)
    if len(extends) > 0 :
        extendHtml = []
        for ext in extends :
            extendHtml.append("[page:{0}] &rarr;".format(ext))
        html.append('		' + " ".join(extendHtml))
        html.append('')
    html.append('		<h1>[name]</h1>')
    html.append('')
    html.append('		<p class="desc">\n\t\t\t{0}\n\t\t</p>'.format(getComment(info)))
    html.append('')
    fileName = getSourceFile(info)
    if kindString == "Class" :
        if "children" in info:
            if isDecorators(info, 'Component') :
                decorators = info['decorators']
                for deco in decorators :
                    name = deco['name']
                    if name == 'Component' :
                        selector = re.compile("selector: '([a-z0-9\-]+)'")
                        matches = selector.findall(deco['arguments']['obj'])
                        if matches :
                            tagName = matches[0]
                            html.append('		<h2>{0}</h2>'.format(deco['name']))
                            html.append('		<code data-type="html">&lt;{0}'.format(tagName))
                            for child in info['children'] :        
                                kindString = child['kindString']
                                if kindString == 'Property' :
                                    attrName = child['name']
                                    if isDecorators(child, 'Input') :
                                        html.append('			[{0}]="..."'.format(attrName))
                                    elif isDecorators(child, 'Output') :
                                        html.append('			({0})="...($event)"'.format(attrName))

                            html.append('		&gt;&lt;/{0}&gt;'.format(tagName))
                            html.append('		</code>')
                            html.append('')

            html.append('		<h2>Properties - Input</h2>')
            if len(extends) > 0 :
                for ext in extends :
                    html.append('		<p>')
                    html.append('			See the base [page:{0} {0}] class for common input properties.'.format(ext))
                    html.append('		</p>')
                html.append('')

            for child in info['children'] :        
                sources = getSourceFile(child)
                kindString = child['kindString']
                if sources == fileName and kindString == 'Property' and isDecorators(child, 'Input') :
                    typeName = getParamTypes(child)
                    isOptional = ''
                    if 'flags' in child and 'isOptional' in child['flags'] and  child['flags']['isOptional'] :
                        isOptional = '?'
                    if typeName.endswith('[]') :
                        typeName = typeName[0:-2]
                        isOptional = '[]' + isOptional

                    html.append('		<h3>[property:{1} {0}]{2}</h3>'.format(
                        child['name'], 
                        typeName,
                        isOptional
                    ))
                    html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format(getComment(child)))
                    html.append('')

            html.append('		<h2>Properties - Output</h2>')
            if len(extends) > 0 :
                for ext in extends :
                    html.append('		<p>')
                    html.append('			See the base [page:{0} {0}] class for common output properties.'.format(ext))
                    html.append('		</p>')
                html.append('')

            for child in info['children'] :        
                sources = getSourceFile(child)
                kindString = child['kindString']
                if sources == fileName and kindString == 'Property' and isDecorators(child, 'Output') :
                    typeName = getParamTypes(child)
                    html.append('		<h3>[property:{1} {0}]</h3>'.format(
                        child['name'], 
                        typeName
                    ))
                    html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format(getComment(child)))
                    html.append('')

            html.append('		<h2>Methods</h2>')
            if len(extends) > 0 :
                for ext in extends :
                    html.append('		<p>')
                    html.append('			See the base [page:{0} {0}] class for common methods.'.format(ext))
                    html.append('		</p>')
                html.append('')
            for child in info['children'] :        
                sources = getSourceFile(child)
                kindString = child['kindString']
                flags = child['flags']
                if sources == fileName and kindString == 'Method' and (
                    ('isPublic' in flags and flags['isPublic']) or ('isProtected' in flags and flags['isProtected'])) :
                    params = []
                    returnType  = 'void' 
                    comment = None
                    if 'signatures' in child :
                        for signatures in child['signatures'] :
                            if 'type' in signatures and 'name' in signatures['type'] :
                                returnType = signatures['type']['name']
                            comment = getComment(signatures)
                            if 'parameters' in signatures :
                                for parameters in signatures['parameters'] :
                                    typeName = getParamTypes(parameters)
                                    isOptional = ''
                                    if 'flags' in child and 'isOptional' in child['flags'] and  child['flags']['isOptional'] :
                                        isOptional = '?'
                                    if typeName.endswith('[]') :
                                        typeName = typeName[0:-2]
                                        isOptional = '[]' + isOptional
                                    params.append(" [param:{0} {1}]{2}".format(typeName, parameters['name'], isOptional)); 
                    html.append('		<h3>[method:{1} {0}]( {2} )</h3>'.format(
                        child['name'], 
                        returnType,
                        ", ".join(params)
                    ))
                    if comment is not None :
                        html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format(comment))
                    html.append('')
    if 'sources' in info :
        sourcesPath = info['sources'][0]['fileName']
        kindString 
        refClassList = getSameSourceFile(getSourceFile(info))
        for mchild in refClassList :
            extends = getExtends(mchild)
            if len(extends) > 0 :
                extendHtml = []
                for ext in extends :
                    extendHtml.append("[page:{0}] &rarr;".format(ext))
                html.append('		' + " ".join(extendHtml))
                html.append('')
            mChildName = mchild['name']
            html.append('		<hr>')
            html.append('		<h1 id="{0}">{0}</h1>'.format(mChildName))
            html.append('')
            html.append('		<p class="desc">\n\t\t\t{0}\n\t\t</p>'.format(getComment(mchild)))
            html.append('')
            fileName = getSourceFile(mchild)
            kindString = mchild["kindString"]
            if kindString == "Interface" :
                html.append('		<h2>Properties</h2>')
                if len(extends) > 0 :
                    for ext in extends :
                        html.append('		<p>')
                        html.append('			See the base [page:{0} {0}] class for common properties.'.format(ext))
                        html.append('		</p>')
                    html.append('')
                if "children" in mchild:
                    for child in mchild['children'] :        
                        sources = getSourceFile(child)
                        kindString = child['kindString']
                        if sources == fileName and kindString == 'Property' :
                            typeName = getParamTypes(child)
                            isOptional = ''
                            if 'flags' in child and 'isOptional' in child['flags'] and  child['flags']['isOptional'] :
                                isOptional = '?'
                            if typeName.endswith('[]') :
                                typeName = typeName[0:-2]
                                isOptional = '[]' + isOptional
                            html.append('		<h3>[property:{1} {0}]{2}</h3>'.format(
                                child['name'], 
                                typeName,
                                isOptional 
                            ))
                            html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format(getComment(child)))
                            html.append('')

            elif kindString == "Type alias" :
                typeName = mchild['type']
                # print("-------------- Type Alias ")
                #types = info['type']['types']
                html.append('')
        
        html.append('		<h2>Source</h2>')
        html.append('		<p>[link:https://github.com/outmindkjg/three-sample/tree/master/src/app/three/{0}.ts {0}.ts]</p>'.format(sourcesPath[14:-3]))
    html.append('	</body>')
    html.append('</html>')
    saveFile('src/assets/ngxapi/en/{0}.html'.format(className),"\n".join(html))
    return True

def getCommentCode(comment : str) :
    # print(comment)
    restComment = comment
    if restComment is not None and restComment.find("```") > -1 :
        while restComment.find("```") > -1 :
            start = restComment.find("```")
            end = restComment.find("```", start + 3)
            blockType = 'text'
            conStart = start + 3
            if restComment[conStart: conStart+2] == 'ts' :
                conStart += 2
                blockType = 'typescript'
            elif restComment[conStart: conStart+4] == 'html' :
                conStart += 4
                blockType = 'html'
            elif restComment[conStart: conStart+4] == 'text' :
                conStart += 4
                blockType = 'text'
            planeHtml = restComment[conStart: end]
            planeHtml = planeHtml.replace('>','&gt;')
            planeHtml = planeHtml.replace('<','&lt;')
            restComment = restComment[0: start] + '<code data-type="'+blockType+'">' +  planeHtml + '</code>' + restComment[end + 3:]
    return restComment

In [14]:
# getGenApiDocs("MeshComponent")
# getGenApiDocs("AbstractGeometryComponent")
getGenApiDocs("GeometryComponent")


True

In [3]:
loadedClass = {}
jsonList = []
for child in simpleApi :
    name = child["name"]
    kindString = child["kindString"]
    if name not in loadedClass and kindString == 'Class':
        loadedClass[name] = child['sources'][0]['fileName']

for name in loadedClass :
    url = loadedClass[name]
    # print(url)
    if url.startswith('src/lib/') :
        if getGenApiDocs(name) :
            jsonList.append('"{0}" : "ngxapi/en/{0}"'.format(name)) 
print(",\n".join(jsonList))

"AbstractChartComponent" : "ngxapi/en/AbstractChartComponent",
"AbstractControllerComponent" : "ngxapi/en/AbstractControllerComponent",
"AbstractGeometryComponent" : "ngxapi/en/AbstractGeometryComponent",
"AbstractMaterialComponent" : "ngxapi/en/AbstractMaterialComponent",
"AbstractObject3dComponent" : "ngxapi/en/AbstractObject3dComponent",
"AbstractObject3dDirective" : "ngxapi/en/AbstractObject3dDirective",
"AbstractSubscribeComponent" : "ngxapi/en/AbstractSubscribeComponent",
"AbstractTextureComponent" : "ngxapi/en/AbstractTextureComponent",
"AbstractThreeController" : "ngxapi/en/AbstractThreeController",
"AbstractThreeDirective" : "ngxapi/en/AbstractThreeDirective",
"AbstractTweenComponent" : "ngxapi/en/AbstractTweenComponent",
"AnglePipe" : "ngxapi/en/AnglePipe",
"AnimationGroupComponent" : "ngxapi/en/AnimationGroupComponent",
"AssetsPipe" : "ngxapi/en/AssetsPipe",
"AudioComponent" : "ngxapi/en/AudioComponent",
"AutoMaterialController" : "ngxapi/en/AutoMaterialController",
"AutoPos